In [1]:
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [2]:
pwd

'/home/peter/code/giadapi/crypto_2/data_preprocessing'

In [3]:
data = pd.read_csv('../data/raw_data/010121_040423_H1experts.csv', lineterminator='\n')

In [4]:
data2 = pd.read_csv('../data/raw_data/010121_040423_H2experts.csv', lineterminator='\n')

In [5]:
data = pd.concat([data, data2])

In [19]:
bitcoin_mask = data['text\r'].str.contains('bitcoin|Bitcoin')

In [21]:
bitcoin_tweets = data[bitcoin_mask]

In [25]:
def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [26]:
bitcoin_tweets['preproc_text'] = bitcoin_tweets['text\r'].apply(preprocess)

/tmp/ipykernel_22441/1012149435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_tweets['preproc_text'] = bitcoin_tweets['text\r'].apply(preprocess)


In [28]:
bitcoin_tweets.shape

(39530, 4)

In [31]:
bitcoin_mask2 = bitcoin_tweets['preproc_text'].str.contains('bitcoin|Bitcoin')

In [33]:
bitcoin_tweets = bitcoin_tweets[bitcoin_mask2]

In [35]:
bitcoin_tweets.drop(columns = 'text\r')

,datetime,username,preproc_text
0,2023-04-03 12:15:20+00:00,saylor,#Bitcoin is Digital Energy.
2,2023-04-02 12:59:03+00:00,saylor,The Robots will only take #Bitcoin.
3,2023-04-01 21:59:22+00:00,saylor,Gen Z Analyst elegantly articulates why #Bitco...
4,2023-04-01 16:45:59+00:00,saylor,#Bitcoin is the Superior Asset.
5,2023-04-01 12:40:19+00:00,saylor,Happy #Bitcoin Fools Day.
...,...,...,...
522702,2021-04-30 03:56:27+00:00,RektProof,$BTC / $USD 2 Different ranges here LTF/HTF ...
522707,2021-04-26 15:03:52+00:00,RektProof,Pretty good start to the year overall. 30R in...
522713,2021-04-24 06:23:55+00:00,RektProof,Power of 3 Another Round? Old low broken and...
522719,2021-04-19 15:48:27+00:00,RektProof,$BTC / $USD Looking for a move into 60-62k th...


In [68]:
data_2 = pd.DataFrame(bitcoin_tweets['username'].value_counts()).reset_index()

In [71]:
data_2.rename(columns = {'index':'usernames', 'username':'tweet_count'}, inplace = True)

In [72]:
data_2

,usernames,tweet_count
0,udiWertheimer,3987
1,PeterMcCormack,2453
2,saylor,1988
3,scottmelker,1978
4,CryptoKaleo,1249
...,...,...
91,cubantobacco,4
92,Vince_Van_Dough,4
93,fishxbt,2
94,IamCryptoWolf,2


In [76]:
bitcoin_tweets.shape

(27455, 4)

In [94]:
key_words = ['Market',
'Short',
'Foo',
'plan',
'Ethereum',
'eth',
'🚀',
'🔥',
'bitcoin',
'BTC',
'HODL',
'collapse',
'crypto',
'cryptocurrency',
'currency',
'price',
'USD',
'dollar',
'scaling',
'scale',
'buy',
'sell',
'dumped',
'dump',
'pump',
'hits',
'greed',
'market',
'cash',
'OnlyUp',
'Stop Loss'
]

key_words_search = ''
for i in key_words:
    key_words_search += f'{i}|'
key_words_search = key_words_search[:-1]

In [95]:
data['text\r'].str.contains(key_words_search).sum()

159584

In [ ]:
data= data[['datetime', 'username', 'text', 'date',
       'negative_bert', 'neutral_bert', 'positive_bert']]

In [ ]:
dup_tweets = data[data['text'].duplicated()]
blacklist = []
for name in dup_tweets['username']:
    blacklist.append(name)

blacklist = list(set(blacklist))

In [ ]:
len(blacklist)

In [ ]:
def to_remove(x):
    if x in blacklist:
        return True
    else:
        return False

In [ ]:
data['to_remove'] = data['username'].apply(to_remove)

In [ ]:
mask = data['to_remove']
data2 = data[~mask]

In [ ]:
data2.shape

In [77]:
bitcoin_tweets.to_csv('expert_bitcoin_tweets.csv')

In [ ]:
#remove users who post supiciouskly large frequency of tweets